## AutoCast processor training example

This notebook demonstrates training a processor directly on encoded data.

### Example dataaset

We use the `ReactionDiffusion` dataset as an example dataset to illustrate training and evaluation of models. This dataset simulates the advection-diffusion equation in 2D.


In [ ]:
from autocast.data.encoded_dataset import MiniWellDataModule
from autocast.metrics import MAE, MSE, RMSE, VRMSE

THE_WELL = False
simulation_name = "rayleigh_benard"
n_steps_input = 1
n_steps_output = 4
stride = 1
rollout_stride = 4


base_path = (
    f"../datasets/{simulation_name}/1e3z5x2c_{simulation_name}_dcae_f32c64_large/cache/{simulation_name}"
)
datamodule = MiniWellDataModule(
    data_path=base_path,
    n_steps_input=n_steps_input, n_steps_output=n_steps_output, stride=stride,
    batch_size=16,
)

### Set-up logging


In [ ]:
from autocast.logging import maybe_watch_model
from autocast.logging.wandb import create_notebook_logger

logger, watch = create_notebook_logger(
    project="autocast-notebooks",
    name=f"06_processor_{simulation_name}_global_cond",
    tags=["notebook", simulation_name],
    enabled=False,
)

In [ ]:
batch = next(iter(datamodule.train_dataloader()))
n_channels = batch.encoded_inputs.shape[-1]
w, h = batch.encoded_inputs.shape[2:4]

### Example shape and batch


In [ ]:
datamodule.train_dataset[0].encoded_inputs.shape

In [ ]:
batch = next(iter(datamodule.train_dataloader()))

batch.encoded_inputs.shape

In [ ]:
from azula.noise import VPSchedule

from autocast.models.processor import ProcessorModel
from autocast.nn.base import TemporalBackboneBase
from autocast.nn.unet import TemporalUNetBackbone
from autocast.nn.vit import TemporalViTBackbone
from autocast.processors.flow_matching import FlowMatchingProcessor

batch = next(iter(datamodule.train_dataloader()))
n_channels = batch.encoded_inputs.shape[-1]
global_cond_channels = (
    batch.global_cond.shape[-1]
    if batch.global_cond is not None
    else None
)

n_latent_in = batch.encoded_inputs.shape[-1]
n_latent_out = batch.encoded_output_fields.shape[-1]

In [ ]:
def get_backbone(backbone_name: str) -> TemporalBackboneBase:
    """Create backbone based on name."""
    if backbone_name == "unet":
        backbone = TemporalUNetBackbone(
            in_channels=n_latent_out,
            out_channels=n_latent_out,
            cond_channels=n_latent_in,
            n_steps_output=n_steps_output,
            n_steps_input=n_steps_input,
            mod_features=200,
            hid_channels=(32, 64, 128),
            hid_blocks=(2, 2, 2),
            spatial=2,
            periodic=False,
        )
    elif backbone_name == "vit":
        backbone = TemporalViTBackbone(
            in_channels=n_latent_out,
            out_channels=n_latent_out,
            cond_channels=n_latent_in,
            n_steps_output=n_steps_output,
            n_steps_input=n_steps_input,
            mod_features=256,
            global_cond_channels=global_cond_channels,
            hid_channels=768,
            # hid_channels=512,
            hid_blocks=12,
            # hid_blocks=8,
            temporal_method="none",
            # temporal_method="attention",
            attention_heads=12,
            # attention_heads=8,
            spatial=2,
            patch_size=1,
            dropout=0.05,
            ffn_factor=4,
            checkpointing=True,
        )
    else:
        raise ValueError(f"Unknown backbone name: {backbone_name}")
    return backbone


backbone_name = "vit"  # set to "unet" or "vit"
backbone = get_backbone(backbone_name)

In [ ]:
from autocast.processors.diffusion import DiffusionProcessor


def get_processor_model(processor_name: str) -> ProcessorModel:
    """Get processor."""
    if processor_name == "flow_matching":
        processor = FlowMatchingProcessor(
            backbone=backbone,
            n_steps_output=n_steps_output,
            n_channels_out=n_latent_out,
            stride=stride,
            flow_ode_steps=25,
        )
    else:
        processor = DiffusionProcessor(
            backbone=backbone,
            schedule=VPSchedule(),
            n_steps_output=n_steps_output,
            n_channels_out=n_latent_out,
        )

    model = ProcessorModel(
        processor=processor,
        learning_rate=1e-4,
        val_metrics=[VRMSE(), MSE(), MAE(), RMSE()],
        test_metrics=[VRMSE(), MSE(), MAE(), RMSE()],
    )
    maybe_watch_model(logger, model, watch)
    return model

processor_name = "flow_matching"  # set to "diffusion" to compare
# processor_name = "diffusion"  # set to "flow_matching" to compare
model = get_processor_model(processor_name)

In [ ]:
model(batch.encoded_inputs, global_cond=batch.global_cond).shape

### Run trainer


In [ ]:
import lightning as L

device = "mps"  # "cpu"
# device = "cuda"  # "cpu"
trainer = L.Trainer(
    max_epochs=10, accelerator=device, logger=logger,
    limit_train_batches=0.2, limit_val_batches=0.2
)
trainer.fit(model, datamodule)
trainer.save_checkpoint(f"./{simulation_name}_{processor_name}_model_fm_0-2.ckpt")

### Run the evaluation


In [ ]:
trainer.test(model, datamodule)